# PART A

# • PROJECT OBJECTIVE: To build a NLP classifier which can use input text parameters to determine the label/s of the blog. 


In [1]:
#Specific to this case 
#study, you can consider the text of the blog: ‘text’ feature as independent variable and ‘topic’ as dependent variable

1. Read and Analyse Dataset. 
A. Clearly write outcome of data analysis(Minimum 2 points)

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity

import nltk
nltk.download('stopwords',quiet=True) # stopword library

nltk.download('punkt', quiet=True) # tokenize library

True

In [3]:
from zipfile import ZipFile
with ZipFile('blogs.zip', 'r') as zip:
    zip.extractall()

In [4]:
df=pd.read_csv("blogtext.csv",nrows=3000)
df.head(5)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [5]:
counts = df.value_counts('topic')
counts

topic
Technology              1607
indUnk                   452
Student                  403
Engineering              119
Education                118
Sports-Recreation         75
InvestmentBanking         70
Non-Profit                46
Science                   33
BusinessServices          21
Internet                  20
Banking                   16
Communications-Media      14
Accounting                 2
Arts                       2
Museums-Libraries          2
dtype: int64

In [6]:
df.isna().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

In [7]:
print(df.shape)


(3000, 7)


In [8]:
df.drop_duplicates(subset="text",inplace=True)

In [9]:
df.text.str.len().describe()

count     2997.000000
mean       878.021688
std       1408.202485
min         20.000000
25%        107.000000
50%        358.000000
75%       1140.000000
max      30737.000000
Name: text, dtype: float64

In [10]:
# Creation of data sub-set
# dropping non value added variables - id & date
df.drop(['id','date'], axis=1, inplace=True)


In [11]:
from langdetect import detect
def detect_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [12]:
df=df[df['text'].apply(detect_english)]
df

,gender,age,topic,sign,text
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,..."
1,male,15,Student,Leo,These are the team members: Drewe...
2,male,15,Student,Leo,In het kader van kernfusie op aarde...
3,male,15,Student,Leo,testing!!! testing!!!
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...
2995,male,35,Technology,Aries,but that zoo exhibit thing was much...
2996,male,35,Technology,Aries,my fave song for the day: Aimee Man...
2997,male,35,Technology,Aries,urlLink America's Best Zoo Exhibit...
2998,male,35,Technology,Aries,'The less one makes declaritive sta...


In [13]:
# converting "age" from int to object
df['age']=df['age'].astype('object')

In [14]:
df.shape


(2813, 5)

In [15]:
df['text'][0] 


'           Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         '

In [16]:
import re
data=df.replace(to_replace ='[^A-Za-z ]', value = '', regex = True)
data

,gender,age,topic,sign,text
0,male,15,Student,Leo,Info has been found pages and MB...
1,male,15,Student,Leo,These are the team members Drewes...
2,male,15,Student,Leo,In het kader van kernfusie op aarde...
3,male,15,Student,Leo,testing testing
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoos Toolbar I can no...
...,...,...,...,...,...
2995,male,35,Technology,Aries,but that zoo exhibit thing was much...
2996,male,35,Technology,Aries,my fave song for the day Aimee Mann...
2997,male,35,Technology,Aries,urlLink Americas Best Zoo Exhibits...
2998,male,35,Technology,Aries,The less one makes declaritive stat...


In [17]:
data['text']

0                  Info has been found   pages and  MB...
1                  These are the team members   Drewes...
2                  In het kader van kernfusie op aarde...
3                              testing  testing          
4                    Thanks to Yahoos Toolbar I can no...
                              ...                        
2995               but that zoo exhibit thing was much...
2996               my fave song for the day Aimee Mann...
2997                urlLink Americas Best Zoo Exhibits...
2998               The less one makes declaritive stat...
2999               While his status as a media persona...
Name: text, Length: 2813, dtype: object

In [18]:
#conversion to lower case

data['text']=data['text'].apply(lambda x: x.lower())
data


,gender,age,topic,sign,text
0,male,15,Student,Leo,info has been found pages and mb...
1,male,15,Student,Leo,these are the team members drewes...
2,male,15,Student,Leo,in het kader van kernfusie op aarde...
3,male,15,Student,Leo,testing testing
4,male,33,InvestmentBanking,Aquarius,thanks to yahoos toolbar i can no...
...,...,...,...,...,...
2995,male,35,Technology,Aries,but that zoo exhibit thing was much...
2996,male,35,Technology,Aries,my fave song for the day aimee mann...
2997,male,35,Technology,Aries,urllink americas best zoo exhibits...
2998,male,35,Technology,Aries,the less one makes declaritive stat...


In [19]:
#stopword removal
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english') 
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (eng_stopwords)]))


In [20]:
#stripping off extra space
data['text'].str.strip()
data

,gender,age,topic,sign,text
0,male,15,Student,Leo,info found pages mb pdf files wait untill team...
1,male,15,Student,Leo,team members drewes van der laag urllink mail ...
2,male,15,Student,Leo,het kader van kernfusie op aarde maak je eigen...
3,male,15,Student,Leo,testing testing
4,male,33,InvestmentBanking,Aquarius,thanks yahoos toolbar capture urls popupswhich...
...,...,...,...,...,...
2995,male,35,Technology,Aries,zoo exhibit thing mucho mucho funny
2996,male,35,Technology,Aries,fave song day aimee mann pavlovs bell album lo...
2997,male,35,Technology,Aries,urllink americas best zoo exhibits
2998,male,35,Technology,Aries,less one makes declaritive statements less apt...


In [21]:
X1=data['text']
Y1=data['topic']
X=data['text']
Y=data['topic']

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(binary=True, ngram_range=(1,2))
X=vectorizer.fit_transform(X)
# check feature names post vectorization
vectorizer.get_feature_names()[:6]


['aa', 'aa anger', 'aa compared', 'aa nice', 'aaa', 'aaa take']

In [23]:
label_counts=dict()

for label in data.topic.values:
    if label in label_counts:
        label_counts[label]+=1
    else:
        label_counts[label]=1

In [24]:
sorted(label_counts.keys())

['Accounting',
 'Arts',
 'Banking',
 'BusinessServices',
 'CommunicationsMedia',
 'Education',
 'Engineering',
 'Internet',
 'InvestmentBanking',
 'MuseumsLibraries',
 'NonProfit',
 'Science',
 'SportsRecreation',
 'Student',
 'Technology',
 'indUnk']

In [25]:
from sklearn.preprocessing import MultiLabelBinarizer
binarizer=MultiLabelBinarizer(classes=sorted(label_counts.keys()))
binarizer

MultiLabelBinarizer(classes=['Accounting', 'Arts', 'Banking',
                             'BusinessServices', 'CommunicationsMedia',
                             'Education', 'Engineering', 'Internet',
                             'InvestmentBanking', 'MuseumsLibraries',
                             'NonProfit', 'Science', 'SportsRecreation',
                             'Student', 'Technology', 'indUnk'])

In [26]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
Y=lb.fit_transform(Y.to_numpy())
Y

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [27]:
X.shape

(2813, 222118)

In [28]:
Y.shape

(2813, 16)

In [29]:
from sklearn.model_selection import train_test_split

Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,random_state=2,test_size = 0.2,stratify=Y)


In [30]:
print(Xtrain.shape)
print(Ytrain.shape)


(2250, 222118)
(2250, 16)


In [31]:
print(Xtest.shape)
print(Ytest.shape)


(563, 222118)
(563, 16)


In [32]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs')
model=OneVsRestClassifier(model)
model.fit(Xtrain,Ytrain)

OneVsRestClassifier(estimator=LogisticRegression())

In [33]:
Ypred=model.predict(Xtest)

In [34]:
Ypred.shape

(563, 16)

In [35]:
pred_probabilities = model.predict_proba(Xtest)
pred_probabilities

array([[3.86951236e-05, 4.54918931e-05, 6.87278995e-04, ...,
        1.19529578e-02, 9.50596213e-01, 1.99334122e-03],
       [2.44857777e-05, 6.08300654e-05, 1.04572415e-03, ...,
        4.01971068e-01, 1.46494609e-01, 7.42771905e-02],
       [2.83984191e-05, 6.68877351e-05, 4.73424779e-04, ...,
        1.11238820e-02, 9.83670788e-01, 1.81941045e-02],
       ...,
       [2.54473194e-05, 5.41185327e-05, 6.84299713e-04, ...,
        1.18382949e-02, 9.80367025e-01, 1.13836518e-02],
       [3.77205480e-05, 6.80826412e-05, 2.64365805e-03, ...,
        1.32191035e-01, 8.64660325e-03, 2.27770208e-02],
       [2.69310418e-05, 5.23744426e-05, 5.92318096e-04, ...,
        1.17841287e-02, 9.59882734e-01, 1.39942917e-02]])

In [36]:
Ypred_inversed = lb.inverse_transform(Ypred)
y_test_inversed = lb.inverse_transform(Ytest)


In [37]:
#roc_auc_ovr

In [38]:
#True Values and Predicted Topic
for i in range(5):
    print('Text:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        Xtest[i],
        y_test_inversed[i],
        Ypred_inversed[i]
    ))

Text:	  (0, 107772)	1
  (0, 110090)	1
  (0, 49849)	1
  (0, 4921)	1
  (0, 193454)	1
  (0, 134035)	1
  (0, 172022)	1
  (0, 39343)	1
  (0, 115621)	1
  (0, 124683)	1
  (0, 147812)	1
  (0, 185797)	1
  (0, 140211)	1
  (0, 167131)	1
  (0, 124506)	1
  (0, 148981)	1
  (0, 166775)	1
  (0, 220143)	1
  (0, 193747)	1
  (0, 79982)	1
  (0, 109771)	1
  (0, 101245)	1
  (0, 142416)	1
  (0, 21932)	1
  (0, 188297)	1
  :	:
  (0, 124612)	1
  (0, 147815)	1
  (0, 132300)	1
  (0, 110462)	1
  (0, 202594)	1
  (0, 124589)	1
  (0, 109826)	1
  (0, 167231)	1
  (0, 151022)	1
  (0, 152767)	1
  (0, 27089)	1
  (0, 99396)	1
  (0, 100192)	1
  (0, 115681)	1
  (0, 114088)	1
  (0, 173647)	1
  (0, 108436)	1
  (0, 117845)	1
  (0, 2182)	1
  (0, 172051)	1
  (0, 188315)	1
  (0, 154448)	1
  (0, 8589)	1
  (0, 6619)	1
  (0, 21944)	1
True labels:	Technology
Predicted labels:	Technology


Text:	  (0, 203910)	1
  (0, 67578)	1
  (0, 30716)	1
  (0, 65893)	1
  (0, 134035)	1
  (0, 220906)	1
  (0, 76064)	1
  (0, 167131)	1
  (0, 47804)	1
  (

In [39]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

def print_evaluation_scores_micro(Ytest, Ypred):
    print('Accuracy score: ', accuracy_score(Ytest, Ypred))
    print('F1 score: ', f1_score(Ytest, Ypred, average='micro'))
    print('Average precision score: ', average_precision_score(Ytest, Ypred, average='micro'))
    print('Average recall score: ', recall_score(Ytest, Ypred, average='micro'))
print_evaluation_scores_micro(Ytest, Ypred)

Accuracy score:  0.5843694493783304
F1 score:  0.6932773109243697
Average precision score:  0.5231102772970727
Average recall score:  0.5861456483126111


In [40]:
from sklearn.metrics import roc_auc_score
classes=sorted(label_counts.keys())
roc_auc_ovr = {}
df_aux = X1.copy()
for i in range(len(classes)):
    # Gets the class
    c = classes[i]
    print(pred_probabilities[:,i])
    df_aux['class'] = Y[i]
    df_aux['prob'] = pred_probabilities[i,:]
    roc_auc_ovr[i] = roc_auc_score(df_aux['class'], df_aux['prob'])
    #roc_auc_ovr[i] = roc_auc_score(Y[i], pred_probabilities[i,:])
    #df_aux = df_aux.reset_index(drop = True)
print("roc score of each topic is",roc_auc_ovr)

[3.86951236e-05 2.44857777e-05 2.83984191e-05 4.76561150e-05
 5.64714782e-05 6.42021996e-05 2.49819287e-05 2.49400480e-05
 2.89127777e-05 2.67701127e-05 2.51264320e-05 2.51756001e-05
 4.44057805e-05 2.93871679e-05 3.65231285e-05 2.61639987e-05
 7.52878861e-05 3.60517728e-05 2.65220984e-05 4.51864146e-05
 3.42349197e-05 4.75478665e-05 2.62676363e-05 3.02321154e-05
 2.85388167e-05 2.78389872e-05 2.84906859e-05 5.98347932e-05
 4.53076511e-05 2.52609191e-05 2.62061637e-05 1.08256629e-04
 2.60950756e-05 2.58681346e-05 2.87599481e-05 5.39303237e-05
 5.31804031e-05 3.27763615e-05 2.53583362e-05 2.93053692e-05
 2.98572750e-05 2.54353917e-05 5.47851706e-05 2.67999707e-05
 4.20800629e-05 3.66475213e-05 3.09593894e-05 3.66575616e-05
 3.35661387e-05 2.83629422e-05 2.67854906e-05 2.58365964e-05
 2.62998310e-05 4.95222370e-05 2.73587986e-05 2.70849510e-05
 2.78230644e-05 3.02355175e-05 1.21486570e-04 9.00078092e-05
 3.98726055e-05 2.65466757e-05 3.69277236e-05 2.97154435e-05
 6.73527538e-05 2.623745

In [41]:
#this is quite good a score with a far less number of sample data
#when we run the classifier with more data from  dataset this might not be the case.
#There are 68,124 records which is large in size to perform analysis and computation, 
#hence  a subset of just 3000 text data samples were taken.


# TF-IDF

In [42]:
# with TF-IDF
## Train_Test Split
from sklearn.feature_extraction.text import TfidfVectorizer
X_train, X_test, y_train, y_test = train_test_split(data['text'].values, Y, test_size=0.15, random_state=42)
print('Training words: {}'.format(X_train.shape[0]))
print('Testing words: {}'.format(X_test.shape[0]))

## TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

## word Selection
from sklearn.feature_selection import SelectKBest, chi2
ch2 = SelectKBest(chi2, k=50)
X_train = ch2.fit_transform(X_train, y_train)
X_test = ch2.transform(X_test)

##Base model Testing
model_tf=LogisticRegression(solver='lbfgs')
model_tf=OneVsRestClassifier(model)
model_tf.fit(X_train,y_train)

y_pred_tf=model_tf.predict(X_test)
def print_evaluation_scores_micro(y_test, y_pred_tf):
    print('Accuracy score: ', accuracy_score(y_test, y_pred_tf))
    print('F1 score: ', f1_score(y_test, y_pred_tf, average='micro'))
    print('Average precision score: ', average_precision_score(y_test, y_pred_tf, average='micro'))
    print('Average recall score: ', recall_score(y_test, y_pred_tf, average='micro'))
print_evaluation_scores_micro(y_test, y_pred_tf)

Training words: 2391
Testing words: 422
Accuracy score:  0.504739336492891
F1 score:  0.5126353790613719
Average precision score:  0.2938131525278393
Average recall score:  0.504739336492891


In [43]:
#Accuracy seemed to have decreased with tf idf vectorisation method, 
#than the count vectorised when used with logistic Regression
#too much preprocessing decreases accuracy as it results in lossof useful words.

In [44]:
#Almost similar set of words from the subset taken are found to be co related or 
#used to find word predictions, word similarities/semantics.
#TF-IDF vectorizer is analyzed for pre-processing techniques and the results are shown with top50 frequently occuring words in the blog  text.

In [45]:
def print_evaluation_scores_macro(Ytest, Ypred):
    print('Accuracy score: ', accuracy_score(Ytest, Ypred))
    print('F1 score: ', f1_score(Ytest, Ypred, average='macro'))
    print('Average precision score: ', average_precision_score(Ytest, Ypred, average='macro'))
    print('Average recall score: ', recall_score(Ytest, Ypred, average='macro'))
print_evaluation_scores_macro(Ytest, Ypred)

Accuracy score:  0.5843694493783304
F1 score:  0.2246150698716006
Average precision score:  nan
Average recall score:  0.187087432784785


C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
def print_evaluation_scores_weighted(Ytest, Ypred):
    print('Accuracy score: ', accuracy_score(Ytest, Ypred))
    print('F1 score: ', f1_score(Ytest, Ypred, average='weighted'))
    print('Average precision score: ', average_precision_score(Ytest, Ypred, average='weighted'))
    print('Average recall score: ', recall_score(Ytest, Ypred, average='weighted'))
print_evaluation_scores_weighted(Ytest, Ypred)

Accuracy score:  0.5843694493783304
F1 score:  0.6094692090047068
Average precision score:  0.5514484927839535
Average recall score:  0.5861456483126111


C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

In [48]:
def build_model_train(Xtrain, Ytrain, X_valid=None, y_valid=None, C=1.0, model='lr'):
    if model=='lr':
        model = LogisticRegression(C=C, penalty='l1', dual=False, solver='liblinear')
        model = OneVsRestClassifier(model)
        model.fit(Xtrain, Ytrain)

    elif model=='svm':
        model = LinearSVC(C=C, penalty='l1', dual=False, loss='squared_hinge')
        model = OneVsRestClassifier(model)
        model.fit(Xtrain, Ytrain)

    elif model=='nbayes':
        model = MultinomialNB(alpha=1.0)
        model = OneVsRestClassifier(model)
        model.fit(Xtrain, Ytrain)

    elif model=='lda':
        model = LinearDiscriminantAnalysis(solver='svd')
        model = OneVsRestClassifier(model)
        model.fit(Xtrain, Ytrain)

    return model

models = ['lr','svm','nbayes']
for model in models:
    model = build_model_train(Xtrain,Ytrain,model=model)
    model.fit(Xtrain,Ytrain)
    Ypred=model.predict(Xtest)
    print("\n")
    print(f"**displaying metrics for the mode {model}\n")
    print_evaluation_scores_micro(Ytest, Ypred)
    print("\n")
    print("\n")
    print_evaluation_scores_macro(Ytest, Ypred)
    print("\n")
    print("\n")
    print_evaluation_scores_weighted(Ytest,Ypred)
    print("\n")
    print("\n")




**displaying metrics for the mode OneVsRestClassifier(estimator=LogisticRegression(penalty='l1',
                                                 solver='liblinear'))

Accuracy score:  0.6216696269982238
F1 score:  0.7310344827586207
Average precision score:  0.5621944702054418
Average recall score:  0.6589698046181173




Accuracy score:  0.6216696269982238
F1 score:  0.3915512274179341
Average precision score:  nan
Average recall score:  0.33030910957394144




Accuracy score:  0.6216696269982238
F1 score:  0.6983398679646712
Average precision score:  0.6021125961894731
Average recall score:  0.6589698046181173






C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1495:



**displaying metrics for the mode OneVsRestClassifier(estimator=LinearSVC(dual=False, penalty='l1'))

Accuracy score:  0.6429840142095915
F1 score:  0.7466666666666666
Average precision score:  0.579430421874601
Average recall score:  0.6962699822380106




Accuracy score:  0.6429840142095915
F1 score:  0.4462381799072446
Average precision score:  nan
Average recall score:  0.4145882979820922




Accuracy score:  0.6429840142095915
F1 score:  0.7200997654058723
Average precision score:  0.6178013493301971
Average recall score:  0.6962699822380106






**displaying metrics for the mode OneVsRestClassifier(estimator=MultinomialNB())

Accuracy score:  0.4191829484902309
F1 score:  0.5472249269717624
Average precision score:  0.33356939425491516
Average recall score:  0.4991119005328597




Accuracy score:  0.4191829484902309
F1 score:  0.20028214069380323
Average precision score:  nan
Average recall score:  0.20531816362227048




Accuracy score:  0.4191829484902309
F1 score:  0.547908

C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vrooky-montzo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1495: